In [1]:

!pip install --upgrade feedparser beautifulsoup4 newspaper3k transformers sentencepiece torch \
                  gTTS moviepy opencv-python-headless tqdm gitpython

# Wav2Lip repo (≈300 MB) – cloned only first time
!git clone https://github.com/Rudrabha/Wav2Lip.git || true
%cd Wav2Lip
# Download pre-trained weights (≈98 MB). Do this once; kept in the repo folder.
!gdown --id 1NbfLxaQ76yvifnWX3voT-yAJ-jjR-Jsp -O wav2lip_gan.pth  # uses gdown; installs automatically
%cd ..


     ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
     -------------------- ------------------- 41.0/81.3 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 81.3/81.3 kB 2.2 MB/s eta 0:00:00
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
     ---------------------------------------- 0.0/211.1 kB ? eta -:--:--
     -------------------- ----------------- 112.6/211.1 kB 3.3 MB/s eta 0:00:01
     ------------------------------------ - 204.8/211.1 kB 2.5 MB/s eta 0:00:01
     -------------------------------------- 211.1/211.1 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
     ---------------------------------------- 0.1/10.5 MB 2.6 MB/s eta 0:00:04
      --------------------------------------- 0.2/10.5 MB 2.1 MB/s eta 0:00:05
     - -------------------------------------- 0.3/10.5 MB 2.5 MB/s eta 0:00:05
     - -------------------------------------- 0.3/10.5 MB 2.4 MB/s eta 0:00:05
    


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\Wav2Lip


Cloning into 'Wav2Lip'...
c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New


'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# ╔═══════════════════════════════════════════════════════════════════╗
# ║  📰 Cell 2 – Define trusted RSS / site list                       ║
# ╚═══════════════════════════════════════════════════════════════════╝
# Feel free to edit / add your own reliable sources.
RSS_FEEDS = [
    "http://feeds.bbci.co.uk/news/rss.xml",
    "https://www.npr.org/rss/rss.php?id=1001",
    "https://feeds.a.dj.com/rss/RSSWorldNews.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/World.xml",
    "https://www.reutersagency.com/feed/?best-topics=world&post_type=best"
]
NUM_STORIES_TO_FETCH_PER_FEED = 5   # lightweight but adjustable


In [6]:
# ╔═══════════════════════════════════════════════════════════════════╗
# ║  🔎 Cell 3 – Fetch latest headlines + full article text           ║
# ╚═══════════════════════════════════════════════════════════════════╝
import feedparser, newspaper, re, textwrap, datetime
from tqdm.notebook import tqdm

def clean_html(text):
    return re.sub("<[^>]+>", "", text or "")

articles = []

for feed_url in RSS_FEEDS:
    parsed = feedparser.parse(feed_url)
    for entry in parsed.entries[:NUM_STORIES_TO_FETCH_PER_FEED]:
        url = entry.link
        try:
            art = newspaper.Article(url)
            art.download(); art.parse()
            full_text = art.text if art.text else clean_html(entry.get("summary", ""))
            articles.append({"title": entry.title, "url": url, "text": full_text})
        except Exception as e:
            print(f"⚠️ Skipping {url[:60]} – {e}")
            
print(f"Collected {len(articles)} articles.")


⚠️ Skipping https://www.bbc.com/news/articles/c1kvw39yv1mo – Article `download()` failed with HTTPSConnectionPool(host='www.bbc.com', port=443): Max retries exceeded with url: /news/articles/c1kvw39yv1mo (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000234C8FED990>, 'Connection to www.bbc.com timed out. (connect timeout=7)')) on URL https://www.bbc.com/news/articles/c1kvw39yv1mo
⚠️ Skipping https://www.bbc.com/news/articles/cp8djlg3zq5o – Article `download()` failed with HTTPSConnectionPool(host='www.bbc.com', port=443): Max retries exceeded with url: /news/articles/cp8djlg3zq5o (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000234C8FEDD80>, 'Connection to www.bbc.com timed out. (connect timeout=7)')) on URL https://www.bbc.com/news/articles/cp8djlg3zq5o
⚠️ Skipping https://www.wsj.com/articles/palestinians-flock-back-to-nort – Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/a

In [11]:
from tqdm.notebook import tqdm
from tqdm import tqdm             # always plain text


In [12]:
# ╔═══════════════════════════════════════════════════════════════════╗
# ║  ✂️ Cell 4 – Per-article abstractive summarisation (BART)         ║
# ╚═══════════════════════════════════════════════════════════════════╝
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch, math

device = 0 if torch.cuda.is_available() else -1   # GPU if you have one
summariser = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    tokenizer="sshleifer/distilbart-cnn-12-6",
    device=device
)

for art in tqdm(articles, desc="Summarising"):
    # 1024 token max – keep chunking simple
    text = art["text"][:4000]
    summary = summariser(text, max_length=80, min_length=25, do_sample=False)[0]["summary_text"]
    art["summary"] = summary


Device set to use cpu




































Summarising: 100%|██████████| 11/11 [00:42<00:00,  3.85s/it]


In [2]:
import torch, platform, sys, os
print("torch:", torch.__version__,
      "| cuda? ", torch.cuda.is_available(),
      "| CUDA devices:", torch.cuda.device_count(),
      "| python:", sys.executable)


torch: 2.3.0+cu118 | cuda?  True | CUDA devices: 1 | python: c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\venv\Scripts\python.exe


In [ ]:

!pip install --upgrade feedparser beautifulsoup4 newspaper3k transformers sentencepiece torch \
                  gTTS moviepy opencv-python-headless tqdm gitpython

# Wav2Lip repo (≈300 MB) – cloned only first time
!git clone https://github.com/Rudrabha/Wav2Lip.git || true
%cd Wav2Lip
# Download pre-trained weights (≈98 MB). Do this once; kept in the repo folder.
!gdown --id 1NbfLxaQ76yvifnWX3voT-yAJ-jjR-Jsp -O wav2lip_gan.pth  # uses gdown; installs automatically
%cd ..


     ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
     -------------------- ------------------- 41.0/81.3 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 81.3/81.3 kB 2.2 MB/s eta 0:00:00
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
     ---------------------------------------- 0.0/211.1 kB ? eta -:--:--
     -------------------- ----------------- 112.6/211.1 kB 3.3 MB/s eta 0:00:01
     ------------------------------------ - 204.8/211.1 kB 2.5 MB/s eta 0:00:01
     -------------------------------------- 211.1/211.1 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
     ---------------------------------------- 0.1/10.5 MB 2.6 MB/s eta 0:00:04
      --------------------------------------- 0.2/10.5 MB 2.1 MB/s eta 0:00:05
     - -------------------------------------- 0.3/10.5 MB 2.5 MB/s eta 0:00:05
     - -------------------------------------- 0.3/10.5 MB 2.4 MB/s eta 0:00:05
    


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\Wav2Lip


Cloning into 'Wav2Lip'...
c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New


'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [5]:

!pip install --upgrade feedparser beautifulsoup4 newspaper3k transformers sentencepiece torch \
                  gTTS moviepy opencv-python-headless tqdm gitpython

# Wav2Lip repo (≈300 MB) – cloned only first time
!git clone https://github.com/Rudrabha/Wav2Lip.git || true
%cd Wav2Lip
# Download pre-trained weights (≈98 MB). Do this once; kept in the repo folder.
!gdown --id 1NbfLxaQ76yvifnWX3voT-yAJ-jjR-Jsp -O wav2lip_gan.pth  # uses gdown; installs automatically
%cd ..


  Using cached torch-2.7.0-cp310-cp310-win_amd64.whl.metadata (29 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.7.0-cp310-cp310-win_amd64.whl (212.5 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)

  Attempting uninstall: sympy

    Found existing installation: sympy 1.13.1

   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
    Uninstalling sympy-1.13.1:
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
      Successfully uninstalled sympy-1.13.1
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   ---------------------------------------- 0/2 [sympy]
   -------------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.0.dev20241112+cu121 requires torch==2.6.0.dev20241112+cu121, but you have torch 2.7.0 which is incompatible.
torchvision 0.20.0.dev20241112+cu121 requires torch==2.6.0.dev20241112+cu121, but you have torch 2.7.0 which is incompatible.


c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\Wav2Lip


fatal: destination path 'Wav2Lip' already exists and is not an empty directory.
'true' is not recognized as an internal or external command,
operable program or batch file.
c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New


usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--fuzzy] [--id] [--proxy PROXY]
             [--speed SPEED] [--no-cookies] [--no-check-certificate]
             [--continue] [--folder] [--remaining-ok] [--format FORMAT]
             [--user-agent USER_AGENT]
             url_or_id
gdown: error: unrecognized arguments: # uses gdown; installs automatically


In [6]:
# ╔═══════════════════════════════════════════════════════════════════╗
# ║  📰 Cell 2 – Define trusted RSS / site list                       ║
# ╚═══════════════════════════════════════════════════════════════════╝
# Feel free to edit / add your own reliable sources.
RSS_FEEDS = [
    "http://feeds.bbci.co.uk/news/rss.xml",
    "https://www.npr.org/rss/rss.php?id=1001",
    "https://feeds.a.dj.com/rss/RSSWorldNews.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/World.xml",
    "https://www.reutersagency.com/feed/?best-topics=world&post_type=best"
]
NUM_STORIES_TO_FETCH_PER_FEED = 5   # lightweight but adjustable


In [4]:
# ╔═══════════════════════════════════════════════════════════════════╗
# ║  🔎 Cell 3 – Fetch latest headlines + full article text           ║
# ╚═══════════════════════════════════════════════════════════════════╝
import feedparser, newspaper, re, textwrap, datetime
from tqdm.notebook import tqdm

def clean_html(text):
    return re.sub("<[^>]+>", "", text or "")

articles = []

for feed_url in RSS_FEEDS:
    parsed = feedparser.parse(feed_url)
    for entry in parsed.entries[:NUM_STORIES_TO_FETCH_PER_FEED]:
        url = entry.link
        try:
            art = newspaper.Article(url)
            art.download(); art.parse()
            full_text = art.text if art.text else clean_html(entry.get("summary", ""))
            articles.append({"title": entry.title, "url": url, "text": full_text})
        except Exception as e:
            print(f"⚠️ Skipping {url[:60]} – {e}")
            
print(f"Collected {len(articles)} articles.")


⚠️ Skipping https://www.wsj.com/articles/palestinians-flock-back-to-nort – Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/articles/palestinians-flock-back-to-northern-gaza-on-foot-after-hostage-release-breakthrough-3f60e2db on URL https://www.wsj.com/articles/palestinians-flock-back-to-northern-gaza-on-foot-after-hostage-release-breakthrough-3f60e2db
⚠️ Skipping https://www.wsj.com/articles/even-chinas-property-stalwart-i – Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/articles/even-chinas-property-stalwart-isnt-immune-from-the-crisis-19799863 on URL https://www.wsj.com/articles/even-chinas-property-stalwart-isnt-immune-from-the-crisis-19799863
⚠️ Skipping https://www.wsj.com/articles/freed-israeli-hostages-still-ha – Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/articles/freed-israeli-hostages-still-had-shrapnel-in-their-bodies-from-oct-7-attack-3de8cd1e o

In [3]:
# ╔═══════════════════════════════════════════════════════════════════╗
# ║  📰 Cell 2 – Define trusted RSS / site list                       ║
# ╚═══════════════════════════════════════════════════════════════════╝
# Feel free to edit / add your own reliable sources.
RSS_FEEDS = [
    "http://feeds.bbci.co.uk/news/rss.xml",
    "https://www.npr.org/rss/rss.php?id=1001",
    "https://feeds.a.dj.com/rss/RSSWorldNews.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/World.xml",
    "https://www.reutersagency.com/feed/?best-topics=world&post_type=best"
]
NUM_STORIES_TO_FETCH_PER_FEED = 5   # lightweight but adjustable


In [9]:
import torch, sys, subprocess, importlib.util, os, platform
print("torch  :", torch.__version__)
print("torch file =", torch.__file__)
print("python :", sys.executable)


torch  : 2.6.0.dev20241112+cu121
torch file = c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\venv\lib\site-packages\torch\__init__.py
python : c:\Users\Harshit Mehra\OneDrive\Desktop\AI DRDO\AI DRDO New\venv\Scripts\python.exe


In [26]:
# ── 1️⃣  FORCE the version string BEFORE Transformers touches torch ──
import torch, re, sys
torch.__version__ = re.sub(r"\.dev.*", "", torch.__version__)   # 2.6.0
assert torch.__version__.startswith("2.6"), torch.__version__

# ── 2️⃣  Load model + tokenizer yourself, guaranteeing .safetensors ──
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

MODEL_ID = "sshleifer/distilbart-cnn-12-6"     # small, public
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_ID,
    use_safetensors=True,      # <-- prevents any .bin download
    torch_dtype="auto",
)

# ── 3️⃣  Build the summariser pipeline from those objects ──
device = 0 if torch.cuda.is_available() else -1
summariser = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_length=90,
    min_length=25,
)
print("✅ summariser ready on", ("GPU" if device==0 else "CPU"))


✅ summariser ready on GPU


In [34]:
import torch, re
torch.__version__ = re.sub(r"\.dev.*", "", torch.__version__)  # 2.6.0


In [3]:
from transformers import pipeline
import torch, re

# ── patch nightly torch version once ──
if ".dev" in torch.__version__:
    torch.__version__ = re.sub(r"\.dev.*", "", torch.__version__)  # "2.6.0"

MODEL_ID = "facebook/bart-large-cnn"        # public, well-known
# or: MODEL_ID = "google/pegasus-xsum"      # also public
# or: MODEL_ID = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # safetensors-only

device = 0 if torch.cuda.is_available() else -1

summariser = pipeline(
    "summarization",
    model=MODEL_ID,
    device=device,
    max_length=90,
    min_length=25,
    # drop use_safetensors flag – transformers will auto-prefer it if present
)

print("✅ loaded", MODEL_ID, "on", "GPU" if device == 0 else "CPU")


Device set to use cuda:0


✅ loaded facebook/bart-large-cnn on GPU


In [5]:
# ╔═══════════  Improved Cell 3 – Robust Scraper  ═══════════╗
import feedparser, newspaper, re, datetime, textwrap, logging
from tqdm.auto import tqdm

RSS_FEEDS = [
    # keep or edit
    "http://feeds.bbci.co.uk/news/rss.xml",
    "https://www.reuters.com/rssFeed/worldNews",
    "http://feeds.bbci.co.uk/news/rss.xml",
    "https://www.npr.org/rss/rss.php?id=1001",
    "https://feeds.a.dj.com/rss/RSSWorldNews.xml",
    "https://rss.nytimes.com/services/xml/rss/nyt/World.xml",
    "https://www.reutersagency.com/feed/?best-topics=world&post_type=best"
    # … remove WSJ / NYT if desired
]

NUM_PER_FEED = 5

# 1️⃣  Set a more “human” user-agent so basic bot checks pass
cfg = newspaper.Config()
cfg.browser_user_agent = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/123.0 Safari/537.36"
)
cfg.request_timeout = 10           # seconds

def clean(html):
    return re.sub("<[^>]+>", "", html or "")

articles = []
skipped  = 0

for feed in RSS_FEEDS:
    for entry in feedparser.parse(feed).entries[:NUM_PER_FEED]:
        url = entry.link
        try:
            art = newspaper.Article(url, config=cfg)
            art.download(); art.parse()
            full = art.text.strip()
            # 2️⃣ Fallback to RSS description if body is empty
            if not full:
                full = clean(entry.get("summary", "")) or entry.title
            articles.append({"title": entry.title, "url": url, "text": full})
        except Exception as e:
            skipped += 1
            logging.warning("⚠️ Skip %s – %s", url[:75], e)
print(f"Collected {len(articles)} good articles, skipped {skipped}.")


Collected 18 good articles, skipped 7.


In [6]:
from tqdm.auto import tqdm
import logging

skipped = 0
for art in tqdm(articles, desc="Summarising"):
    try:
        text = art["text"] or art["title"]
        art["summary"] = summariser(text, truncation=True)[0]["summary_text"]
    except Exception as e:
        skipped += 1
        logging.warning("⚠️  Fallback to title for %s  (%s)", art["url"], e)
        art["summary"] = art["title"]

print(f"Done.  Summaries created: {len(articles)-skipped}, fall-backs: {skipped}")


Summarising:   0%|          | 0/18 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
)
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
)
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
)
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For deb

Done.  Summaries created: 1, fall-backs: 17


In [7]:
#cell 5 – Print the final bulletin
# ╔═══════════════════════════════════════════════════════════════════╗
# ║  📜 Cell 5 – Print the final bulletin                             ║
# ╚═══════════════════════════════════════════════════════════════════╝
from datetime import datetime

top_articles = sorted(
    [a for a in articles if a.get("summary")],
    key=lambda a: len(a["text"]),
    reverse=True
)[:7]

final_bulletin = "\n".join(
    f"{i}. {a['summary']}" for i, a in enumerate(top_articles, 1)
)
print("----- Today’s Bulletin -----\n")
print(final_bulletin)


----- Today’s Bulletin -----

1. Inside Ukraine's last maternity ward in a region surrounded by Russian forces
2. Russia hits Ukraine with the largest drone-and-missile attack of the war so far
3. Zelensky says 'US silence' over Russian attacks encourages Putin
4. Zelensky says 'US silence' over Russian attacks encourages Putin
5. Mumbai's iconic pav bread might soon be toast
6. Renationalised railways not on track for cheaper fares
7. Renationalised railways not on track for cheaper fares


In [8]:
# ╔═══════════════════════════════════════════════════════════════════╗
# ║  🗣️ Cell 6 – Text-to-speech with gTTS → MP3                      ║
# ╚═══════════════════════════════════════════════════════════════════╝
from gtts import gTTS
from pathlib import Path

tts = gTTS(text=final_bulletin, lang="en", tld="com")  # change accent with tld (“co.uk”, “co.in”, etc.)
AUDIO_PATH = Path("bulletin.mp3")
tts.save(AUDIO_PATH)
print(f"Saved narration → {AUDIO_PATH}")


Saved narration → bulletin.mp3


In [12]:
# ╔═══════════  Cell 7 – Lip-sync with Wav2Lip  ═══════════╗
import subprocess, shlex, os
from pathlib import Path

# 0️⃣  Clone repo once
if not Path("Wav2Lip").exists():
    !git clone https://github.com/Rudrabha/Wav2Lip.git

# 1️⃣  Ensure weight file is present  (choose Option A or Option B above)
# … insert the Option A gdown block here, or do the manual step once …

# 2️⃣  Check anchor face
ANCHOR_IMG = "anchor.jpg"
assert Path(ANCHOR_IMG).exists(), "❗ Add a clear photo named anchor.jpg"

# 3️⃣  Run inference
AUDIO_PATH   = Path("bulletin.mp3")
OUTPUT_VIDEO = "anchor_narration.mp4"

cmd = f"""
python Wav2Lip/inference.py \
  --checkpoint_path Wav2Lip/wav2lip_gan.pth \
  --face {ANCHOR_IMG} \
  --audio {AUDIO_PATH} \
  --outfile {OUTPUT_VIDEO}
"""
print("Running Wav2Lip …")
subprocess.run(shlex.split(cmd), check=True)
print("🎥  Lip-synced clip →", OUTPUT_VIDEO)


Running Wav2Lip …


CalledProcessError: Command '['python', 'Wav2Lip/inference.py', '--checkpoint_path', 'Wav2Lip/wav2lip_gan.pth', '--face', 'anchor.jpg', '--audio', 'bulletin.mp3', '--outfile', 'anchor_narration.mp4']' returned non-zero exit status 1.

In [13]:
import subprocess, shlex, textwrap, sys, os, pathlib

cmd = f"""
python Wav2Lip/inference.py \
  --checkpoint_path Wav2Lip/wav2lip_gan.pth \
  --face anchor.jpg \
  --audio bulletin.mp3 \
  --outfile anchor_narration.mp4
"""
print("⏳  Running Wav2Lip with full log …\n")
completed = subprocess.run(
    shlex.split(cmd),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,   # merge stderr → stdout
    text=True
)
print(completed.stdout)


⏳  Running Wav2Lip with full log …

Using cpu for inference.
Number of frames available for inference: 1
Extracting raw audio...
ffmpeg version N-119683-ga18b2c2696-20250524 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.1.0 (crosstool-NG 1.27.0.42_35c1e72)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libfribidi --enable-gmp --enable-libxml2 --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libfreetype --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac